In [2]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  

In [3]:
pair = "USDCHF"
NB_H1_CANDLES = 20000

In [4]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [5]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2019-12-31 03:00:00,0.96859,0.96866,0.96758,0.96761,0.96761,0.96859,2.777778,93.518519,90.740741,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-12-31 04:00:00,0.96760,0.96763,0.96736,0.96760,0.96760,0.96760,88.888889,88.888889,0.000000,...,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-12-31 05:00:00,0.96761,0.96786,0.96748,0.96771,0.96761,0.96771,34.210526,60.526316,26.315789,...,90.740741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-12-31 06:00:00,0.96772,0.96789,0.96766,0.96786,0.96772,0.96786,26.086957,86.956522,60.869565,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-12-31 07:00:00,0.96788,0.96788,0.96759,0.96764,0.96764,0.96788,17.241379,100.000000,82.758621,...,26.315789,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2019-12-31 03:00:00,0.96859,0.96866,0.96758,0.96761,0.96761,0.96859,2.777778,93.518519,90.740741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-31 04:00:00,0.96760,0.96763,0.96736,0.96760,0.96760,0.96760,88.888889,88.888889,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-31 05:00:00,0.96761,0.96786,0.96748,0.96771,0.96761,0.96771,34.210526,60.526316,26.315789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-31 06:00:00,0.96772,0.96789,0.96766,0.96786,0.96772,0.96786,26.086957,86.956522,60.869565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-31 07:00:00,0.96788,0.96788,0.96759,0.96764,0.96764,0.96788,17.241379,100.000000,82.758621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,2023-03-23 07:00:00,0.91522,0.91562,0.91492,0.91562,0.91522,0.91562,42.857143,100.000000,57.142857,...,0.919234,0.925779,0.912689,0.001871,0.918363,0.920559,0.916167,-0.002048,-0.001792,-0.000256
19995,2023-03-23 08:00:00,0.91559,0.91600,0.91407,0.91522,0.91522,0.91559,59.585492,78.756477,19.170984,...,0.918878,0.925514,0.912242,0.001909,0.918064,0.920494,0.915634,-0.002046,-0.001843,-0.000203
19996,2023-03-23 09:00:00,0.91521,0.91663,0.91376,0.91606,0.91521,0.91606,50.522648,80.139373,29.616725,...,0.918539,0.925141,0.911938,0.002004,0.917873,0.920677,0.915069,-0.001953,-0.001865,-0.000088
19997,2023-03-23 10:00:00,0.91605,0.91668,0.91190,0.91391,0.91391,0.91605,42.050209,86.820084,44.769874,...,0.918105,0.924656,0.911554,0.001694,0.917496,0.921158,0.913834,-0.002030,-0.001898,-0.000132


In [7]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST'],
      dtype='object')

In [8]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00011

In [10]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,KeLo,MACD,SIGNAL_MACD,HIST,low_perv,high_perv,low_next,high_next,pivots_l,pivots_h
0,2020-01-03 13:00:00,0.97342,0.97393,0.97321,0.97350,0.97342,0.97350,29.166667,40.277778,11.111111,...,0.969125,0.000583,0.000291,0.000293,0.97322,0.97425,0.97218,0.97379,False,False
1,2020-01-03 14:00:00,0.97350,0.97379,0.97218,0.97255,0.97255,0.97350,22.981366,81.987578,59.006211,...,0.969204,0.000593,0.000357,0.000236,0.97321,0.97393,0.97226,0.97314,True,False
2,2020-01-03 15:00:00,0.97255,0.97314,0.97226,0.97292,0.97255,0.97292,32.954545,75.000000,42.045455,...,0.969267,0.000621,0.000414,0.000208,0.97218,0.97379,0.97204,0.97330,False,False
3,2020-01-03 16:00:00,0.97288,0.97330,0.97204,0.97204,0.97204,0.97288,0.000000,66.666667,66.666667,...,0.969209,0.000570,0.000447,0.000123,0.97226,0.97314,0.97016,0.97257,False,True
4,2020-01-03 17:00:00,0.97206,0.97257,0.97016,0.97060,0.97060,0.97206,18.257261,78.838174,60.580913,...,0.968827,0.000413,0.000440,-0.000026,0.97204,0.97330,0.97007,0.97128,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19945,2023-03-23 05:00:00,0.91585,0.91611,0.91448,0.91569,0.91569,0.91585,74.233129,84.049080,9.815951,...,0.914682,-0.002011,-0.001644,-0.000366,0.91561,0.91730,0.91512,0.91601,True,False
19946,2023-03-23 06:00:00,0.91569,0.91601,0.91512,0.91522,0.91522,0.91569,11.235955,64.044944,52.808989,...,0.915964,-0.002063,-0.001728,-0.000335,0.91448,0.91611,0.91492,0.91562,False,False
19947,2023-03-23 07:00:00,0.91522,0.91562,0.91492,0.91562,0.91522,0.91562,42.857143,100.000000,57.142857,...,0.916167,-0.002048,-0.001792,-0.000256,0.91512,0.91601,0.91407,0.91600,False,False
19948,2023-03-23 08:00:00,0.91559,0.91600,0.91407,0.91522,0.91522,0.91559,59.585492,78.756477,19.170984,...,0.915634,-0.002046,-0.001843,-0.000203,0.91492,0.91562,0.91376,0.91663,False,False


In [11]:
BUY = 1
SELL = -1
NONE = 0

# df_an['BUY_SIG'] = 0
# df_an['SELL_SIG'] = 0
df_an['SIGNAL'] = 0
# df_an['SIGNAL_LABEL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

In [12]:
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
#                     df_an.BUY_SIG[i] = 1
                    df_an.SIGNAL[i] = BUY
#                     df_an.SIGNAL_LABEL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] #- SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
#                     df_an.SELL_SIG[i] = 1
                    df_an.SIGNAL[i] = SELL
#                     df_an.SIGNAL_LABEL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_12224\715545850.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = BUY
C:\Users\meysam\AppData\Local\Temp\ipykernel_12224\715545850.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.low[i-1] #- SPREAD
C:\Users\meysam\AppData\Local\Temp\ipykernel_12224\715545850.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close[i]

In [14]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [15]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'low_perv', 'high_perv', 'low_next', 'high_next',
       'pivots_l', 'pivots_h', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [ ]:
# df_features = df_an#[feature_col]
# mu = df_features.mean()
# std = df_features.std()
# df_features = (df_features-mu)/std

In [16]:
X = []
Y = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
#     'pivots_h','pivots_h'
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces = []
for i in range(4,len(df_an)):
    if df_an.SIGNAL[i] != NONE:
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an[fc][i])

        for fc in feature_col:
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [18]:
print(len(X))
print(len(Y))
print(len(indeces))

3333
3333
3333


In [26]:
print(len(feature))

202


In [20]:
SPLIT_INDEX = int(0.8*len(X))

In [21]:
# X_train = X[:8000]
# Y_train = Y[:8000]
# X_test = X[8000:]
# Y_test = Y[8000:]

X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [25]:
print(len(X_train))
print(len(Y_train))
print(len(indeces_train))
print(len(X_test))
print(len(Y_test))
print(len(indeces_test))

2666
2666
2666
667
667
667


In [ ]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [ ]:
# df_an.SIGNAL[9530:].value_counts()

In [ ]:
# help(MLPClassifier)

In [29]:
clf = MLPClassifier( solver='adam', alpha=1e-5,learning_rate_init=1e-4,
                     tol=0.0000010, hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,verbose=True,warm_start = True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [30]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 2.56492595
Iteration 2, loss = 1.21894354
Iteration 3, loss = 0.71270211
Iteration 4, loss = 0.54538343
Iteration 5, loss = 0.47268934
Iteration 6, loss = 0.42989640
Iteration 7, loss = 0.40731242
Iteration 8, loss = 0.38518167
Iteration 9, loss = 0.36550018
Iteration 10, loss = 0.35134247
Iteration 11, loss = 0.33764298
Iteration 12, loss = 0.32471880
Iteration 13, loss = 0.31485460
Iteration 14, loss = 0.30443293
Iteration 15, loss = 0.29554566
Iteration 16, loss = 0.28513151
Iteration 17, loss = 0.28059663
Iteration 18, loss = 0.27541434
Iteration 19, loss = 0.27084703
Iteration 20, loss = 0.26525913
Iteration 21, loss = 0.25559918
Iteration 22, loss = 0.25819027
Iteration 23, loss = 0.24826614
Iteration 24, loss = 0.23823946
Iteration 25, loss = 0.23363622
Iteration 26, loss = 0.23010789
Iteration 27, loss = 0.22186480
Iteration 28, loss = 0.21799895
Iteration 29, loss = 0.21488295
Iteration 30, loss = 0.21269153
Iteration 31, loss = 0.21276310
Iteration 32, los

Iteration 254, loss = 0.01673791
Iteration 255, loss = 0.01658559
Iteration 256, loss = 0.01665610
Iteration 257, loss = 0.01699072
Iteration 258, loss = 0.01707571
Iteration 259, loss = 0.01566731
Iteration 260, loss = 0.01508304
Iteration 261, loss = 0.01479853
Iteration 262, loss = 0.01403353
Iteration 263, loss = 0.01495534
Iteration 264, loss = 0.01454894
Iteration 265, loss = 0.01436247
Iteration 266, loss = 0.01372431
Iteration 267, loss = 0.01323864
Iteration 268, loss = 0.01369513
Iteration 269, loss = 0.01324193
Iteration 270, loss = 0.01355801
Iteration 271, loss = 0.01280297
Iteration 272, loss = 0.01282117
Iteration 273, loss = 0.01252728
Iteration 274, loss = 0.01377433
Iteration 275, loss = 0.01348391
Iteration 276, loss = 0.01271754
Iteration 277, loss = 0.01214291
Iteration 278, loss = 0.01187652
Iteration 279, loss = 0.01155329
Iteration 280, loss = 0.01153114
Iteration 281, loss = 0.01163561
Iteration 282, loss = 0.01121624
Iteration 283, loss = 0.01188110
Iteration 

Iteration 503, loss = 0.00124144
Iteration 504, loss = 0.00122578
Iteration 505, loss = 0.00119205
Iteration 506, loss = 0.00115159
Iteration 507, loss = 0.00116248
Iteration 508, loss = 0.00115568
Iteration 509, loss = 0.00112668
Iteration 510, loss = 0.00116344
Iteration 511, loss = 0.00109405
Iteration 512, loss = 0.00108927
Iteration 513, loss = 0.00108396
Iteration 514, loss = 0.00107072
Iteration 515, loss = 0.00108792
Iteration 516, loss = 0.00107515
Iteration 517, loss = 0.00106752
Iteration 518, loss = 0.00118461
Iteration 519, loss = 0.00116145
Iteration 520, loss = 0.00113423
Iteration 521, loss = 0.00108632
Iteration 522, loss = 0.00103438
Iteration 523, loss = 0.00100467
Iteration 524, loss = 0.00097156
Iteration 525, loss = 0.00099678
Iteration 526, loss = 0.00101385
Iteration 527, loss = 0.00094629
Iteration 528, loss = 0.00095122
Iteration 529, loss = 0.00093859
Iteration 530, loss = 0.00094102
Iteration 531, loss = 0.00092624
Iteration 532, loss = 0.00091248
Iteration 

Iteration 752, loss = 0.00014528
Iteration 753, loss = 0.00014173
Iteration 754, loss = 0.00014135
Iteration 755, loss = 0.00013560
Iteration 756, loss = 0.00014126
Iteration 757, loss = 0.00013493
Iteration 758, loss = 0.00013803
Iteration 759, loss = 0.00013830
Iteration 760, loss = 0.00013468
Iteration 761, loss = 0.00013374
Iteration 762, loss = 0.00012881
Iteration 763, loss = 0.00013012
Iteration 764, loss = 0.00012785
Iteration 765, loss = 0.00012656
Iteration 766, loss = 0.00013399
Iteration 767, loss = 0.00013267
Iteration 768, loss = 0.00012596
Iteration 769, loss = 0.00012408
Iteration 770, loss = 0.00012198
Iteration 771, loss = 0.00012512
Iteration 772, loss = 0.00012136
Iteration 773, loss = 0.00012800
Iteration 774, loss = 0.00013175
Iteration 775, loss = 0.00012785
Iteration 776, loss = 0.00012013
Iteration 777, loss = 0.00011544
Iteration 778, loss = 0.00012042
Iteration 779, loss = 0.00012186
Iteration 780, loss = 0.00012530
Iteration 781, loss = 0.00011363
Iteration 

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(200, 200),
              learning_rate_init=0.0001, max_iter=100000, random_state=1,
              tol=1e-06, verbose=True, warm_start=True)

In [31]:
y_perdict = clf.predict(X_test)

In [32]:
confusion_matrix(Y_test, y_perdict)

array([[336,  19],
       [ 28, 284]], dtype=int64)

In [33]:
(Y_test==y_perdict).sum()/len(y_perdict)

0.9295352323838081

In [36]:
print(len(indeces_test))
print(len(y_perdict))
# y_perdict

667
667


In [37]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'low_perv', 'high_perv', 'low_next', 'high_next',
       'pivots_l', 'pivots_h', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [39]:
indeces_test[0]

16014

In [60]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL']].iloc[indeces_test[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_12224\2659789318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL.loc[idx] = y_perdict[i]


In [61]:
df_test

,time,open,high,low,close,TP,SL,SIGNAL
0,2022-08-03 03:00:00,0.95832,0.95896,0.95778,0.95840,0.0,0.0,0
1,2022-08-03 04:00:00,0.95839,0.95850,0.95708,0.95719,0.0,0.0,0
2,2022-08-03 05:00:00,0.95719,0.95734,0.95601,0.95619,0.0,0.0,0
3,2022-08-03 06:00:00,0.95619,0.95635,0.95453,0.95461,0.0,0.0,0
4,2022-08-03 07:00:00,0.95460,0.95509,0.95412,0.95482,0.0,0.0,0
...,...,...,...,...,...,...,...,...
3936,2023-03-23 05:00:00,0.91585,0.91611,0.91448,0.91569,0.0,0.0,0
3937,2023-03-23 06:00:00,0.91569,0.91601,0.91512,0.91522,0.0,0.0,0
3938,2023-03-23 07:00:00,0.91522,0.91562,0.91492,0.91562,0.0,0.0,0
3939,2023-03-23 08:00:00,0.91559,0.91600,0.91407,0.91522,0.0,0.0,0


In [64]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51233 entries, 0 to 51232
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    51233 non-null  datetime64[ns]
 1   open    51233 non-null  float64       
 2   high    51233 non-null  float64       
 3   low     51233 non-null  float64       
 4   close   51233 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.0 MB


In [85]:
df_test[df_test.SIGNAL !=0]

,time,open,high,low,close,TP,SL,SIGNAL
5,2022-08-03 08:00:00,0.95483,0.95613,0.95476,0.95606,0.95994,0.95412,1
12,2022-08-03 15:00:00,0.95877,0.96017,0.95849,0.95955,0.96481,0.95692,1
15,2022-08-03 18:00:00,0.96423,0.96484,0.96312,0.96322,0.95950,0.96508,-1
17,2022-08-03 20:00:00,0.96258,0.96295,0.96152,0.96164,0.96218,0.96137,1
32,2022-08-04 11:00:00,0.96126,0.96178,0.96059,0.96083,0.95805,0.96222,-1
...,...,...,...,...,...,...,...,...
3882,2023-03-20 23:00:00,0.92678,0.92880,0.92671,0.92868,0.92716,0.92944,-1
3886,2023-03-21 03:00:00,0.92865,0.92920,0.92855,0.92902,0.93068,0.92819,1
3892,2023-03-21 09:00:00,0.93136,0.93152,0.92909,0.92977,0.92595,0.93168,-1
3899,2023-03-21 16:00:00,0.92368,0.92495,0.92354,0.92362,0.91988,0.92549,-1


In [82]:
df_an.loc[indeces_train[-1]]

time                2022-08-03 01:00:00
open                            0.95754
high                            0.95754
low                             0.95651
close                           0.95739
body_lower                      0.95739
body_upper                      0.95754
body_bottom_perc              85.436893
body_top_perc                     100.0
body_perc                     14.563107
direction                            -1
body_size                       0.00015
low_change                      0.02196
high_change                   -0.005221
body_size_change                  -75.0
mid_point                      0.957025
mid_point_prev_2                0.95722
body_size_prev                   0.0006
direction_prev                      1.0
direction_prev_2                    1.0
body_perc_prev                46.511628
body_perc_prev_2                   30.0
HANGING_MAN                         1.0
SHOOTING_STAR                       0.0
SPINNING_TOP                        0.0


In [66]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [67]:
df_res_m5 = gt.run_test()

run_test...
Result: 1050.0


In [68]:
# orginal 
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_12224\2236611890.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin)


In [75]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_12224\729023029.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
df_res_m5

,running,start_index_m5,profit_factor,loss_factor,SPREAD,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,balance,time
0,False,4233,2,-1.0,0.00011,0.95617,0.96025,1,0.95994,0.95412,2.0,2022-08-03 13:20:00,2022-08-03 09:00:00,1.059707e+02,2022-08-03 13:20:00
1,False,4317,2,-1.0,0.00011,0.95966,0.96507,1,0.96481,0.95692,2.0,2022-08-03 17:15:00,2022-08-03 16:00:00,1.122478e+02,2022-08-03 17:15:00
2,False,4377,2,-1.0,0.00011,0.96175,0.96227,1,0.96218,0.96137,2.0,2022-08-03 21:35:00,2022-08-03 21:00:00,1.168558e+02,2022-08-03 21:35:00
3,False,4353,2,-1.0,0.00011,0.96322,0.95951,-1,0.95950,0.96508,2.0,2022-08-04 12:25:00,2022-08-03 19:00:00,1.238483e+02,2022-08-04 12:25:00
4,False,4555,2,-1.0,0.00011,0.96083,0.95802,-1,0.95805,0.96222,2.0,2022-08-04 15:40:00,2022-08-04 12:00:00,1.313594e+02,2022-08-04 15:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,False,50548,2,-1.0,0.00011,0.92913,0.93085,1,0.93068,0.92819,2.0,2023-03-21 08:15:00,2023-03-21 04:00:00,4.042984e+14,2023-03-21 08:15:00
638,False,50620,2,-1.0,0.00011,0.92977,0.92564,-1,0.92595,0.93168,2.0,2023-03-21 12:35:00,2023-03-21 10:00:00,4.305248e+14,2023-03-21 12:35:00
639,False,49880,2,-1.0,0.00011,0.92922,0.92315,-1,0.92318,0.93224,2.0,2023-03-21 17:10:00,2023-03-16 19:00:00,4.564846e+14,2023-03-21 17:10:00
640,False,50704,2,-1.0,0.00011,0.92362,0.91926,-1,0.91988,0.92549,2.0,2023-03-22 20:00:00,2023-03-21 17:00:00,4.884141e+14,2023-03-22 20:00:00


In [77]:
df_res_m5.result.value_counts()

 2.0    564
-1.0     78
Name: result, dtype: int64

In [78]:
564/(78+564)

0.8785046728971962

In [80]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])